In [76]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import json
import pymongo
from selenium.webdriver.remote.webelement import WebElement
from IPython import display
from base64 import b64decode
import sys 
import time

In [77]:
pic_cache_path = "E:\\workspace\\movie_center_ui\\cache\\javbus_pic_cache"
root_url = "https://www.javbus.com"
user_data_path = "E:\\workspace\\movie_center_ui\\cache\\chromium_cache_80\\user-data"
disk_cache_path = "E:\\workspace\\movie_center_ui\\cache\\chromium_cache_80\\disk-cache"
chrome_exe_path = "E:\\workspace\\movie_center_ui\\chrome-win-80\\chrome.exe"
chrome_driver_path = "E:\\workspace\\movie_center_ui\\selenium_scraper\\chromedriver_800397.exe"
proxy_server_address = "http://localhost:7890"
chrome_options=Options()
# chrome_options.add_argument("--headless") #设置chrome浏览器无界面模式
chrome_options.binary_location = chrome_exe_path
chrome_options.add_argument("user-data-dir=%s" % user_data_path)
chrome_options.add_argument("disk-cache-dir=%s" % disk_cache_path)
chrome_options.add_argument("window-size=1000,1000")
chrome_options.add_argument("proxy-server=%s" % proxy_server_address)
chrome_options.add_argument("--disable-web-security")

#建立浏览器实例
browser = webdriver.Chrome(options=chrome_options, executable_path=chrome_driver_path)
browser.set_script_timeout(120)
# 开始请求
# browser.get(root_url)
# 数据库
client = pymongo.MongoClient('mongodb://localhost:27017')
db = client.movieTest

In [78]:
def eprint(*args, **kwargs):
  # 把内容输出到错误流中，stdout可以输出函数的返回值给调用的nodejs？
  print(*args, file=sys.stderr, **kwargs)
  sys.stderr.flush()  # 让node的stream可以emit事件

def display_element(element: WebElement):
  # 预览一个web元素的缩略图
  # 只有在一个块的最后一个输出时 才会出现
  return display.Image(b64decode(element.screenshot_as_base64))
  
def get_brick_info(element: WebElement):
  photo_img_ele = element.find_element_by_xpath("//div[@class='photo-frame']/img")
  photo_img_src = photo_img_ele.get_attribute("src")
  title = photo_img_ele.get_attribute("title")
  date_tags = element.find_elements_by_tag_name("date")
  serial = date_tags[0].get_attribute("innerHTML").strip()
  date = date_tags[1].get_attribute("innerHTML").strip()
  info_page_herf = element.get_attribute("href")
  
  ret = {
    'title':title,
    'date':date,
    'serial':serial,
    'info_page_href':info_page_herf,
    'photo_img_src':photo_img_src 
  }
#   print(ret)
  return ret

# _info_tags = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p")
# 
# serial = _info_tags[0].find_element_by_xpath("./span[2]").get_attribute("innerHTML").strip()
# print(_info_tags)
# print(serial)

In [79]:
def find_jav_search_record(_serial):
  # 去数据库中查找_serial有无被查找过 和 最近的查找时间  返回 number
  # 没有找过 就放入一个 返回 0
  search_record_collection = db['javbus_search_record']
  one = search_record_collection.find_one({
    "serial": _serial
  })
  if one is None:
    c = int(time.time())
    search_record_collection.insert_one({
      "serial": _serial,
      "t": c,
    })
    return 0
  else:
    return one['t']
 
# brick{ serial, info_page_href}
# 加载失败会返回None，加载成功的找到内容的话返回数据库的_id
def movie_insert_update(db, browser: webdriver.Chrome, _brick):
  # 查看有没有记录
  movie_collection = db['javbus_movie']
  one = movie_collection.find_one({
    "serial": _brick['serial'],
  })
  if not one is None:
    return str(one['_id'])
  # 查找之前有无查询记录
  search_t = find_jav_search_record(_brick['serial'])
  if (search_t!=0):
    # 说明以前找过 而且javbus中没有记录 就没有记录到db中
    eprint('serial had found and not found:[%s]' % _brick['serial'])
    return;
  
  # 进入网页
  browser.get(_brick['info_page_href'])
  
  # 等信息加载完
  _info_tags = []
  try:
    xpath = "//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p"
    locator = (By.XPATH,xpath)
    WebDriverWait(browser, 10).until(EC.presence_of_element_located(locator))
    _info_tags = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p")
  except Exception as e:
    eprint('page load fail:[%s]' % browser.current_url,e)
    # 设置查询记录
    # TODO
    return;
    
  # 开始爬取信息
  
  # 识别码 发行日期
  serial = _info_tags[0].find_element_by_xpath("./span[2]").get_attribute("innerHTML").strip()
  release_date = _info_tags[1].text.split(": ")[1].strip()
 
 # 三个带有链接的信息 信息还可能不存在
  tags = browser.find_elements_by_xpath\
    ("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p/a/preceding-sibling::*")
  tag_texts = ["導演:","製作商:","發行商:"]
  studio = {
    'name': '',
    'href': '',
  }
  lab = {
    'name': '',
    'href': '',
  }
  director = {
    'name': '',
    'href': '',
  }
  for i,tag in enumerate(tags):
    tag_text = tag.text
    a_tag = browser.find_elements_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/p/a")[i]
    if tag_text == tag_texts[0]:
      # 导演
      director['name'] = a_tag.text.strip()
      director['href'] = a_tag.get_attribute("href")
    if tag_text == tag_texts[1]:
      # studio
      studio['name'] = a_tag.text.strip()
      studio['href'] = a_tag.get_attribute("href")
    if tag_text == tag_texts[2]:
      # lab
      lab['name'] = a_tag.text.strip()
      lab['href'] = a_tag.get_attribute("href")
   
  # 演员列表
  # 暂无演出者列表的时候 要特殊处理 这个时候 就没有p标签了！！
  jav_idols = []
  category_p_tag_index = -3 # 有演员列表时 类别是倒数第3个 没有时 是倒数第2个
  try:
    # 有 idol 列表时 有ul标签
    browser.find_element_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class, 'info')]/ul")
  except NoSuchElementException:
    category_p_tag_index = -2
  else:
    jav_idol_tags = _info_tags[-1].find_elements_by_xpath("./span/a")
    jav_idols = [{
      'name': x.text.strip(),
      'href': x.get_attribute('href')
    } for x in jav_idol_tags]
    
  # 类别
  category_tags = _info_tags[category_p_tag_index].find_elements_by_xpath("./span/a")
  categories = [{
    'name': x.text.strip(),
    'href': x.get_attribute("href"),
  } for x in category_tags]
  
  # 磁力连接
  magnet_tags = browser.find_elements_by_xpath("//table[@id='magnet-table']/tr/td/a[1]")
  magnets = [
    {
    'magnet_link': magnet_tags[x].get_attribute('href'),
    'size': magnet_tags[x+1].text.strip(),
    'date': magnet_tags[x+2].text.strip(),
  } for x in range(0,len(magnet_tags),3) ]
  
  # 样品图像
  sample_tags = browser.find_elements_by_xpath("//div[@id='sample-waterfall']/a")
  samples = [{
    'name': "%s-sample-pic-%d.%s" % (serial, i,x.get_attribute("href").split('.')[-1]),
    'href': x.get_attribute('href')
  } for i,x in enumerate(sample_tags)]
  
  # 封面
  cover_href = browser.find_element_by_xpath("//div[contains(@class, 'movie')]/div[contains(@class,'screencap')]/a/img")\
    .get_attribute("src")
  cover_pic_name = "%s-cover-pic.%s" % (serial, cover_href.split('.')[-1])
  
  # 向数据库中插入数据
  for i, id in enumerate(category_insert_update(db, categories)):
    categories[i]['id'] = id
  for i, id in enumerate(jav_idol_insert_update(db, jav_idols)):
    jav_idols[i]['id'] = id
  if studio['name'] != '':
    studio['id'] = studio_insert_update(db, studio)
  if lab['name'] != '':
    lab['id'] = lab_insert_update(db, lab)
  if director['name'] != '':
    director['id'] = director_insert_update(db, director)
  
  # 下载图片
  download_pic(cover_href, cover_pic_name)
  for sample in samples:
    download_pic(sample['href'], sample['name'])
  
  # 插入主页面
  one = movie_collection.insert_one({
    'serial':serial,
    'cover': cover_pic_name,
    'release_date': release_date,
    'studio': studio,
    'lab': lab,
    'director': director,
    'category': categories,
    'idol': jav_idols,
    'magnet': magnets,
    'sample_pic': samples,
  })
  eprint(one.inserted_id)
  #设置查询记录
  #TODO
  return one.inserted_id

def category_insert_update(db, categories):
  category_collection: pymongo.collection.Collection = db['javbus_category']
  ret = []
  for category in categories:
    name = category['name']
    one = category_collection.find_one({"name": name})
    if one is None:
      ret.append(category_collection.insert_one(category).inserted_id)
    else:
      ret.append(str(one['_id']))
  return ret

def jav_idol_insert_update(db, idols):
  idol_collection = db['javbus_idol']
  ret = []
  for idol in idols:
    name = idol['name']
    one = idol_collection.find_one({"name":name})
    if one is None:
      ret.append(idol_collection.insert_one(idol).inserted_id)
    else:
      ret.append(str(one['_id']))
  return ret

def studio_insert_update(db, studio):
  studio_collection = db['javbus_studio']
  one = studio_collection.find_one({"name":studio['name']})
  if one is None:
    return studio_collection.insert_one(studio).inserted_id
  else:
    return str(one['_id'])

def lab_insert_update(db, lab):
  lab_collection = db['javbus_lab']
  one = lab_collection.find_one({"name":lab['name']})
  if one is None:
    return lab_collection.insert_one(lab).inserted_id
  else:
    return str(one['_id'])

def director_insert_update(db, director):
  director_collection = db['javbus_director']
  one = director_collection.find_one({"name": director['name']})
  if one is None:
    return director_collection.insert_one(director).inserted_id
  else:
    return str(one['_id'])
  
def download_pic(src, filename, dst_path=pic_cache_path):
  bytes_str = browser.execute_async_script("""
    var callback = arguments[arguments.length -1];
    var src = arguments[arguments.length-2];
    
    xhr = new XMLHttpRequest()
    xhr.responseType = "arraybuffer"
    xhr.open('GET',src)
    xhr.send()
    xhr.onreadystatechange = () => {
      if (xhr.readyState === XMLHttpRequest.DONE) {
        callback(new Uint8Array(xhr.response))
      }
    }
  """, src)
  raw_data = bytes(list(bytes_str))
  f = open("%s/%s" % (dst_path, filename), 'wb') # TODO 用操作系统系统的字符串拼接
  f.write(raw_data)
  f.close()
# movie_insert_update(db, browser)

# 向后爬取count页的信息
def scrapy_pages(count):
  for _count in range(count):
    
    # 找到页面内所有的 砖头
    _locator = (By.XPATH, "/html/body/div[@class='container-fluid']/div/div[@id='waterfall']/div/div")
    WebDriverWait(browser, 10).until(EC.presence_of_all_elements_located(_locator))
    _bricks = browser.find_elements_by_xpath("/html/body/div[@class='container-fluid']/div/div[@id='waterfall']/div/div/a[@class='movie-box']")
    _bricks = [get_brick_info(brick) for brick in _bricks]
    
    if len(browser.window_handles) == 1:
      # 新建窗口
      browser.execute_script("""
        window.open('about:blank','_blank');
      """)
    # 焦点转到第二个窗口上
    browser.switch_to.window(browser.window_handles[1])
    
    # 在第二个窗口上爬取详情页面
    for _brick in _bricks:
      movie_insert_update(db,browser,_brick)
      
    # 返回页码窗口
    browser.switch_to.window(browser.window_handles[0])

    # 翻页了！！ 
    _locator = (By.ID,"next")
    WebDriverWait(browser, 10).until(EC.presence_of_element_located(_locator))
    next_tag = browser.find_element_by_id("next")
    # next_tag.click()   # 很奇怪  元素不出现在可视窗口中 就不能点击
    browser.get(next_tag.get_attribute('href'))
    eprint('goto page %s' % browser.current_url)

def scrapy_page_by_page():
  browser.switch_to_window(browser.window_handles[0])
  try:
    scrapy_pages(50)
  except TimeoutException as e:
    eprint(e.msg)
    eprint(e.stacktrace)
  finally:
    eprint('done')

def scrapy_by_serial_no(serial_no_file_path):
  # 从json文件中读入需要爬取的番号列表
  # 输出成功保存的个数
  with open(serial_no_file_path, 'r') as f:
    _json_str = f.readline()
    _serial_no_list = json.loads(_json_str)
    ret = []
    for _s in _serial_no_list:  # _s["id"] 保存的是番号 是前端页面组合好的字符串
      # 调用方法把页面保存入数据库
      _ = movie_insert_update(db,browser, {
        "serial": _s["id"],
        "info_page_href": root_url+"/"+_s["id"]
      })
      ret.append(_)
    ret = list(filter(lambda x: not (x is None), ret))
    print(len(ret))

In [80]:
def scrapy_by_idol(idol_id):
  # https://www.javbus.com/star/{idol_id}
  eprint('goto scrapy idol: %s' % idol_id)
  browser.get('%s/star/%s'%(root_url, idol_id))
  while True:
    try:
      scrapy_pages(1)
    except TimeoutException as e:
      eprint(e.msg)
      eprint(e.stacktrace)
      eprint('go next page fail..')
      break;


# 件再用nodejs调用文件，让浏览器和python连在一起


# sys.argv 第一个参数是当前python在运行的py文件的文件路径
# 第二个为了突出是从bus station 调用的, 就要识别为 --busstation 才能使用功能，
# 不然的话 就认为程序运行在ipynb中，是在调试模式中，就不执行功能
if sys.argv[1]=='--busstation' :
  if sys.argv[2]=='--movie-import-from-file':
    _json_file_path = sys.argv[3]  # 默认值是 r"E:\workspace\movie_center_ui\tmp_serialno.json"
    scrapy_by_serial_no(_json_file_path)
  if sys.argv[2]=='--scrap-page-by-page':
    scrapy_page_by_page()
  if sys.argv[2]=='--scrap-idol':
    idol_id = sys.argv[3]
    scrapy_by_idol(idol_id)
else:
  ########################################################################### 开发区 在 ipynb中开发新功能 和 调试bug
  # scrapy_by_serial_no(r"E:\workspace\movie_center_ui\tmp_serialno.json")
  scrapy_by_idol("okq") # https://www.javbus.com/star/ow0

# TODO
# 2. 捕捉错误 记录错误的url 重新爬取


##%% 结束
browser.close()





goto scrapy idol: okq


{'title': '三上悠亜', 'date': '2023-01-06', 'serial': 'SSIS-570', 'info_page_href': 'https://www.javbus.com/SSIS-570', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2022-11-04', 'serial': 'SSIS-541', 'info_page_href': 'https://www.javbus.com/SSIS-541', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2022-09-09', 'serial': 'OFJE-377', 'info_page_href': 'https://www.javbus.com/OFJE-377', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2022-09-09', 'serial': 'SSIS-509', 'info_page_href': 'https://www.javbus.com/SSIS-509', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2022-08-05', 'serial': 'SSIS-477', 'info_page_href': 'https://www.javbus.com/SSIS-477', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2022-07-08', 'serial': 'SSIS-448', 'info_page_href': 'https://www.ja

64074f5789708dbfec690c04
64074f5e89708dbfec690c06
64074f6689708dbfec690c08
64074f7089708dbfec690c0a
64074f7a89708dbfec690c0c
64074f8389708dbfec690c0e
64074f8e89708dbfec690c10
64074f9a89708dbfec690c14
64074fa689708dbfec690c17
64074fb389708dbfec690c19
64074fc189708dbfec690c1b
64074fca89708dbfec690c1d
64074fd489708dbfec690c1f
64074fe389708dbfec690c21
64074fed89708dbfec690c23
64074ff889708dbfec690c25
6407500689708dbfec690c27
goto page https://www.javbus.com/star/okq/2


{'title': '三上悠亜', 'date': '2021-06-30', 'serial': 'SIVR-137', 'info_page_href': 'https://www.javbus.com/SIVR-137', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2021-06-16', 'serial': 'OFJE-318', 'info_page_href': 'https://www.javbus.com/OFJE-318', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2021-06-16', 'serial': 'SSIS-088', 'info_page_href': 'https://www.javbus.com/SSIS-088', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2021-05-14', 'serial': 'SSIS-062', 'info_page_href': 'https://www.javbus.com/SSIS-062', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2021-04-14', 'serial': 'SSIS-037', 'info_page_href': 'https://www.javbus.com/SSIS-037', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2021-03-18', 'serial': 'OFJE-304', 'info_page_href': 'https://www.ja

6407501b89708dbfec690c29
6407502689708dbfec690c2b
6407503389708dbfec690c2d
6407503d89708dbfec690c2f
6407504b89708dbfec690c31
6407505789708dbfec690c33
6407506589708dbfec690c35
6407507289708dbfec690c37
6407508589708dbfec690c39
6407509689708dbfec690c3b
640750a789708dbfec690c3d
640750b289708dbfec690c3f
640750bb89708dbfec690c41
640750c689708dbfec690c43
640750d189708dbfec690c45
goto page https://www.javbus.com/star/okq/3


...

640751fc89708dbfec690c75
6407520689708dbfec690c77
6407521189708dbfec690c79
6407522089708dbfec690c7b
6407522d89708dbfec690c7d
6407523b89708dbfec690c7f
6407524589708dbfec690c81
6407525d89708dbfec690c83
6407526989708dbfec690c85
6407527289708dbfec690c87
6407527789708dbfec690c89
6407527f89708dbfec690c8b
6407528b89708dbfec690c8d
6407529489708dbfec690c8f
6407529f89708dbfec690c91
640752ad89708dbfec690c93
640752b789708dbfec690c95
640752c589708dbfec690c98
640752d389708dbfec690c9a
640752ea89708dbfec690c9c
640752f689708dbfec690c9e
640752fc89708dbfec690ca0
6407530989708dbfec690ca2
goto page https://www.javbus.com/star/okq/5


{'title': '三上悠亜', 'date': '2018-10-18', 'serial': 'OFJE-170', 'info_page_href': 'https://www.javbus.com/OFJE-170', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2018-09-19', 'serial': 'OFJE-167', 'info_page_href': 'https://www.javbus.com/OFJE-167', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2018-09-19', 'serial': 'SSNI-301', 'info_page_href': 'https://www.javbus.com/SSNI-301', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2018-09-07', 'serial': 'OFJE-165', 'info_page_href': 'https://www.javbus.com/OFJE-165', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2018-08-16', 'serial': 'SSNI-279', 'info_page_href': 'https://www.javbus.com/SSNI-279', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2018-08-04', 'serial': 'OFJE-161', 'info_page_href': 'https://www.ja

6407531d89708dbfec690ca5
6407532c89708dbfec690ca8
6407533789708dbfec690caa
6407535a89708dbfec690cac
6407536589708dbfec690cae
6407537789708dbfec690cb0
6407539889708dbfec690cb2
640753a489708dbfec690cb4
640753b589708dbfec690cb6
640753c289708dbfec690cb8
640753ce89708dbfec690cba
640753de89708dbfec690cbc
640753eb89708dbfec690cbe
640753f489708dbfec690cc0
640753fc89708dbfec690cc2
6407540689708dbfec690cc4
6407541289708dbfec690cc6
6407541e89708dbfec690cc8
6407542889708dbfec690cca
6407543189708dbfec690ccc
6407543a89708dbfec690cce
6407544489708dbfec690cd0
6407544d89708dbfec690cd2
6407545789708dbfec690cd4
6407546389708dbfec690cd6
6407546d89708dbfec690cd8
goto page https://www.javbus.com/star/okq/6


{'title': '三上悠亜', 'date': '2017-06-17', 'serial': 'SNIS-940', 'info_page_href': 'https://www.javbus.com/SNIS-940', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2017-05-14', 'serial': 'SNIS-919', 'info_page_href': 'https://www.javbus.com/SNIS-919', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2017-05-09', 'serial': 'SIVR-002', 'info_page_href': 'https://www.javbus.com/SIVR-002', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2017-05-02', 'serial': 'SIVR-001', 'info_page_href': 'https://www.javbus.com/SIVR-001', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2017-04-25', 'serial': 'SIVR-003', 'info_page_href': 'https://www.javbus.com/SIVR-003', 'photo_img_src': 'https://www.javbus.com/pics/actress/okq_a.jpg'}
{'title': '三上悠亜', 'date': '2017-04-15', 'serial': 'SNIS-896', 'info_page_href': 'https://www.ja

6407548a89708dbfec690cda
6407549589708dbfec690cdc
6407549f89708dbfec690cde
640754a689708dbfec690ce0
640754ae89708dbfec690ce2
640754b989708dbfec690ce4
640754c389708dbfec690ce6
640754cf89708dbfec690cea
640754da89708dbfec690cec
640754e489708dbfec690cee
640754e789708dbfec690cf0
640754eb89708dbfec690cf2
640754ee89708dbfec690cf4
640754f589708dbfec690cf6
640754f889708dbfec690cf8
640754fb89708dbfec690cfa
640754ff89708dbfec690cfc

None
go next page fail..


In [84]:
# scrapy_by_idol("p4o")
# scrapy_by_idol("nj9")
# scrapy_by_idol("2yl")
# scrapy_by_idol("p8y")
# scrapy_by_idol("pw2")
# scrapy_by_idol("n5q")
# scrapy_by_idol("2ks")
# scrapy_by_idol("pmv")
# scrapy_by_idol("vkq")
# scrapy_by_idol("s3m")
# scrapy_by_idol("305")
# scrapy_by_idol("un8")
# scrapy_by_idol("7y7")
# scrapy_by_idol("2eg")
# scrapy_by_idol("wfo")
# scrapy_by_idol("vbn")

goto scrapy idol: p4o


{'title': 'RION', 'date': '2022-11-11', 'serial': 'MKCK-321', 'info_page_href': 'https://www.javbus.com/MKCK-321', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2021-12-10', 'serial': 'OFJE-341', 'info_page_href': 'https://www.javbus.com/OFJE-341', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2021-09-24', 'serial': 'OFJE-331', 'info_page_href': 'https://www.javbus.com/OFJE-331', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2021-06-16', 'serial': 'OFJE-318', 'info_page_href': 'https://www.javbus.com/OFJE-318', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2021-03-18', 'serial': 'OFJE-304', 'info_page_href': 'https://www.javbus.com/OFJE-304', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2021-01-16', 'serial': 'OFJE-294', 'info_page_href': 'https://www.ja

6409f34c89708dbfec691582
6409f35589708dbfec691584
6409f36689708dbfec691586
6409f37989708dbfec691588
6409f38789708dbfec69158f
6409f39589708dbfec691591
6409f3a589708dbfec691593
6409f3b189708dbfec691595
6409f3bf89708dbfec691597
6409f3ce89708dbfec691599
6409f3dd89708dbfec69159b
goto page https://www.javbus.com/star/p4o/2


{'title': 'RION', 'date': '2018-04-07', 'serial': 'OFJE-143', 'info_page_href': 'https://www.javbus.com/OFJE-143', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2018-04-07', 'serial': 'OFJE-144', 'info_page_href': 'https://www.javbus.com/OFJE-144', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2018-03-15', 'serial': 'SSNI-151', 'info_page_href': 'https://www.javbus.com/SSNI-151', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2018-02-15', 'serial': 'OFJE-138', 'info_page_href': 'https://www.javbus.com/OFJE-138', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2018-02-15', 'serial': 'SSNI-126', 'info_page_href': 'https://www.javbus.com/SSNI-126', 'photo_img_src': 'https://www.javbus.com/pics/actress/p4o_a.jpg'}
{'title': 'RION', 'date': '2018-01-13', 'serial': 'SSNI-100', 'info_page_href': 'https://www.ja

6409f3fd89708dbfec69159d
6409f40b89708dbfec69159f
6409f41989708dbfec6915a1
6409f42889708dbfec6915a3
6409f43689708dbfec6915a5
6409f44089708dbfec6915a7
6409f44c89708dbfec6915a9
6409f45e89708dbfec6915ab
6409f46e89708dbfec6915ad
6409f47f89708dbfec6915af
6409f48a89708dbfec6915b1
6409f4ae89708dbfec6915b3
6409f4ce89708dbfec6915b5
6409f4de89708dbfec6915b7
6409f50389708dbfec6915b9
6409f51489708dbfec6915bb
6409f52289708dbfec6915bd
6409f53189708dbfec6915c1
6409f53d89708dbfec6915c3
6409f54b89708dbfec6915c5
6409f55c89708dbfec6915c7
goto page https://www.javbus.com/star/p4o/3


...

640a4add89708dbfec6929f6

None
go next page fail..
goto scrapy idol: vbn


{'title': '目黒めぐみ', 'date': '2023-01-21', 'serial': 'NSFS-158', 'info_page_href': 'https://www.javbus.com/NSFS-158', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2022-06-17', 'serial': 'PPBD-237', 'info_page_href': 'https://www.javbus.com/PPBD-237', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2022-04-15', 'serial': 'PPBD-233', 'info_page_href': 'https://www.javbus.com/PPBD-233', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2022-02-11', 'serial': 'PPBD-229', 'info_page_href': 'https://www.javbus.com/PPBD-229', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2022-02-11', 'serial': 'PPBD-228', 'info_page_href': 'https://www.javbus.com/PPBD-228', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2021-10-15', 'serial': 'PPBD-221', 'info_page_href': 'https://

640a4af389708dbfec6929f8
640a4af589708dbfec6929fb
640a4af889708dbfec6929fd
640a4b0089708dbfec6929ff
640a4b0789708dbfec692a06
640a4b1189708dbfec692a09
640a4b1689708dbfec692a0b
640a4b1e89708dbfec692a0d
640a4b2889708dbfec692a0f
640a4b3189708dbfec692a11
640a4b3789708dbfec692a14
640a4b3e89708dbfec692a16
640a4b4689708dbfec692a18
640a4b4c89708dbfec692a1a
goto page https://www.javbus.com/star/vbn/2


{'title': '目黒めぐみ', 'date': '2019-12-28', 'serial': 'JUL-103', 'info_page_href': 'https://www.javbus.com/JUL-103', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2019-12-14', 'serial': 'PPPD-815', 'info_page_href': 'https://www.javbus.com/PPPD-815', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2019-12-07', 'serial': 'MEYD-554', 'info_page_href': 'https://www.javbus.com/MEYD-554', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2019-11-14', 'serial': 'PPPD-802', 'info_page_href': 'https://www.javbus.com/PPPD-802', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}
{'title': '目黒めぐみ', 'date': '2019-10-12', 'serial': 'PPPD-793', 'info_page_href': 'https://www.javbus.com/PPPD-793', 'photo_img_src': 'https://www.javbus.com/pics/actress/vbn_a.jpg'}



None
go next page fail..
